<a href="https://colab.research.google.com/github/marceloraimundo/csv/blob/main/Atentados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

# **Modelizacion de atentados en el gran Buenos Aires durante 1959**

---


Mi idea fue modelizar una serie de atentados ocurridos durante 1959, año que forma parte del proceso conocido como "Resistencia Peronista", en 18 ciudades del gran Buenos Aires.

Los datos corresponden a los contenidos en un legajo de la Dirección de Inteligencia de la Policia de Buenos Aires (DIPBA). Para mas información pueden consultar: https://programminghistorian.org/es/lecciones/visualizacion-y-animacion-de-tablas-historicas-con-R

*Problema planteado*: Si bien este legajo es bastante completo, cuando se recurre a otra documentación, como los periódicos, surgen diferencias con la información de espioneje, comunmente un subregistro por parte de la policía, es decir en el diario podemos encontrar que se les escapan atentados. Lo que me propuse fue crear un modelo al que pueda ingresarle fechas no registradas y me estime si fue probable un atentado en determinado día, como estrategia para hacer una busqueda dirigida en los periódicos que permita encontrar posibles subregistros por parte de la policía, sin necesidad de revisar toda la colección de periodicos de 1959.



---



Comienzo instalando los distintos paquetes a utilizar



In [1]:
# Aquí importo los paquetes que voy a usar de Keras
from keras.models import Sequential
from keras.layers import Dense

# Scaling y train test split desde SciKit
from sklearn.model_selection import train_test_split

# Evaluación
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

# Manejo de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
%matplotlib inline




---



Importo datos en formato cvs desde GitHub (que no pide permiso como al usar Gdrive)



In [2]:
# importo csv con datos desde GitHub
at1959=pd.read_csv('https://raw.githubusercontent.com/marceloraimundo/csv/main/Atentadxs%2059.csv')




---


El dataframe contiene 3 columnas: Fecha, Ciudad y Atentado

Fecha: en formato dd/mm/aaaa, ciudad en string y atentado es 0/1 según si no hubo o hubo algun atentado en dicha ciudad durante esa fecha

Nota: *En cantidad de casos es un dataset muy pequeño, pero decidí usarlo ya que encontre en la web casos de modelos de redes neuronales probados sobre pequeños conjunto de datos (estudios sobre diabetes y cancer)*



In [ ]:
print(at1959)

Nota: *Para realizar un procesamiento adecuado en sintonía con el sentido del seminario de redes neuronales, debí modificar la tabla inicial de datos, ya que el legajo policial incluye sólo positivos (cuando hubo atentado). Lo que resolví para esta ocasión es sumar a los 265 casos positivos, 57 casos repartidos bastante proporcionalemente (en los meses y en las ciudades) que incluyen fechas donde no hubo atentados (o sea atentado=0). Si hubiera sido mejor generar los casos nuevos con algún calculo aleatorio, es una duda que me queda.*



---


Como los datos corresponden solo a un año y ademas no se abordaron cuestiones
referentes a series temporales (tema que no manejo) tome otra desición: las fechas no las tomo como tales, sino que a partir de ellas genere dos vectores, uno con el número del día y el otro con el número de mes.
Para hacerlo elegí la función DatatimeIndex, con la idea de capturar el día y el mes pero no como entero sino como un índice, pensando que con ello evito confundir a la red (los enteros pueden atribuir proximidad, no se si lo resolví correctamente...)





In [4]:
# Hago un vector con días y meses 

dia = pd.DatetimeIndex(at1959['fecha']).day
mes = pd.DatetimeIndex(at1959['fecha']).month




---


Como las ciudades son datos categóricos y la red debe recibir como entrada números, realicé un One Hot Encoding para convertir los valores






In [ ]:
# Hago one hot para la variable categorica
ciudades = pd.get_dummies(at1959.ciudad)
print(ciudades)




---


Luego concatené dia, mes y ciudad en un dataframe llamado at59

In [ ]:
# Concateno vectores en dataframe y modifico nombre columnas
at59 = pd.concat([pd.DataFrame(dia),pd.DataFrame(mes), ciudades], axis=1)
at59.columns.values[0]="dia"
at59.columns.values[1]="mes"
print(at59)
# Guardo el nombre de las columnas para luego poder crear dataframe de datos nuevos 'x_test'
at59_columns=list(at59.columns.values)



---


Posteriormente, con los datos de la columna objetivo que sería Atentado (toma valor 0 si no hubo en esa fecha y lugar, o valor 1 si hubo), creé el vector target59


---



In [7]:
# Defino los resultados
target59 = at1959.atentado



---

Una vez que termino con la de adaptar los datos para que sean cargables en la red, tomo la libreria train_test_split de SciKit para dividir el conjunto de datos en uno de entrenamiento y otro de test (elegí un 33%)




In [8]:
# Split del dataframe
at59_train, at59_test, target59_train, target59_test = train_test_split(at59,target59,test_size=0.33,random_state=101) 



---

Ahora paso a crear y trabajar con el **modelo de red** (tome uno de los modelos dados en el seminario)


---



In [9]:
# Creo el modelo
model = Sequential()
model.add(Dense(12, input_dim=20, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
# Compilo el modelo - Salida binaria: hubo o no hubo atentado en determinada fecha y lugar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entreno el modelo
history = model.fit(at59_train, target59_train, validation_data=(at59_test,target59_test), epochs=50, batch_size=10)

In [ ]:
# Hago la grafica de pérdida y precisión
plt.plot(history.history['loss'], label='MAE (training data)')
plt.plot(history.history['val_loss'], label='MAE (validation data)')
plt.title('MAE')
plt.ylabel('Valor MAE')
plt.xlabel('Epoca')
plt.legend(loc="upper left")
plt.show()



---

**Evaluación y predicción** - tomando el mismo conjunto de test


---



In [ ]:
# Evalua el modelo / con el conjunto entero de datos....
scores = model.evaluate(at59, target59)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Predicciones sobre el conjunto de test
predictions = model.predict(at59_test)

print('MAE: ',mean_absolute_error(target59_test,predictions))
print('MSE: ',mean_squared_error(target59_test,predictions))
print('RMSE: ',np.sqrt(mean_squared_error(target59_test,predictions)))
print('Variance Regression Score: ',explained_variance_score(target59_test,predictions))





---

# **Predicción utilizando otro set de datos**

La realizo con un set de datos de otra procedencia, un conjunto de 5 atentados, pero extraidos de otra fuente (Diario El Día)

*Nota*: Aquí tuve que codificar estos datos realizando operaciones que me permitan recuperar el total de categorias correspondientes al total de ciudades en juego (son 18), pues si usaba directamente el mismo metodo de mas arriba con un set de test mas pequeño, en este caso 5 atentados en 3 ciudades, construyo un dataframe con menos columnas que las definidas en la capa de entrada de la red (está compilada para recibir 20 columnas y si hacia el get_dummies directo sobre el nuevo set solo genera 5 columnas)


---



In [17]:
# Importo csv con datos nuevos para predecir desde GitHub (fuente: Diario El Día - Mayo 1959)
at1959_eldia=pd.read_csv('https://raw.githubusercontent.com/marceloraimundo/csv/main/Atentadxs%2059_ElDia.csv')

In [ ]:
print(at1959_eldia)

In [19]:
# Creo dataframe x_test (que es el que construyo para probar prediccion) tomando nombres de columnas de at59
x_test=pd.DataFrame(columns=at59_columns) 

# Almaceno los resultados para luego chequear las predicciones
target59_eldia = at1959_eldia.atentado

# Nuevamente hago un vector con días y meses 
dia_test = pd.DatetimeIndex(at1959_eldia['fecha']).day
mes_test = pd.DatetimeIndex(at1959_eldia['fecha']).month

# También hago one hot para la variable categorica
ciudades_test = pd.get_dummies(at1959_eldia.ciudad)

# Me quedo con los nombres de las columnas del dataframe nuevo
nom_city=ciudades_test.columns.values

# asigno vectores dia y mes al x_test
x_test.dia = dia_test
x_test.mes = mes_test


In [20]:
# Aca reemplazo las columnas de ciudades del x_test con las que vienen de ciudades de at1959_eldia
for i in range (len(nom_city)):
    x_test.loc[:,[nom_city[i]]]=ciudades_test.loc[:,nom_city[i]]

# Pongo a 0 las Nan
x_test=x_test.fillna(0)

# Este es el dataset que puedo mandar a la red
at59_eldia_test = x_test




---

**Evaluación y predicción** 


---



In [ ]:
# Evaluo el modelo / con los datos nuevos de otro origen
scores = model.evaluate(at59_eldia_test, target59_eldia)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Predicciones sobre los datos de otro origen

predictions = model.predict(at59_eldia_test)

print('MAE: ',mean_absolute_error(target59_eldia,predictions))
print('MSE: ',mean_squared_error(target59_eldia,predictions))
print('RMSE: ',np.sqrt(mean_squared_error(target59_eldia,predictions)))
print('Variance Regression Score: ',explained_variance_score(target59_eldia,predictions))

# Resultados Predicción 
for i in range (len(predictions)):
      print ('Caso ', i, '--> Resultado ', np.round(predictions[i]))



---


# **Conclusión parcial**

Pienso que el modelo no es muy bueno, mas allá que hizo predicciones 'acertadas' con la informacion del dataset de origen extra-policial.

Si bien no entiendo mucho de estadistica, valores de R2 rondando 0 parece que no habla muy bien del modelo. El *loss* me parece muy alto, aunque se combina con una buena *accuracy*


## **Posibles soluciones**

Realizar pruebas variando los hiperparámetros (hice algunas, variando cantidad de neuronas, agregando/sacando capas, cambiando epocas y lotes, usando otros optimizers, pero no hubo variaciones notables respecto loss, accuracy, Mae y R2)

Aumentar el dataset de entrenamiento y validación usando algun metodo de los que comentaron los docentes en clase (K-Fold)

En ambos casos, necesitaría una guia breve por parte de los docentes, pues se me dificulta hacerlo por mi cuenta

Desde ya gracias por todo, Marcelo



---

